# 🔬 HBV 그룹 비교 분석

**목적**: 건강인 vs HBV 환자 면역세포 분포 비교

---

## 📋 분석 내용

| STEP | 내용 |
|------|------|
| 1 | 환경 설정 및 데이터 로드 |
| 2 | 그룹별 세포 분포 비교 |
| 3 | 세포 유형별 비율 분석 |
| 4 | 탈진 마커 발현 분석 |
| 5 | 차별 발현 유전자 분석 |
| 6 | 통계적 유의성 검정 |
| 7 | 결과 시각화 및 저장 |

⚠️ **사전 조건**: `Step1_HBV_tahoe_analysis.ipynb`를 먼저 실행

In [ ]:
# @title STEP 1: 환경 설정 및 데이터 로드
import os, sys
from google.colab import drive
drive.mount('/content/drive')

PROJECT_PATH = '/content/drive/MyDrive/tahoe-x1'
ANALYSIS_PATH = f'{PROJECT_PATH}/viral_hepatitis/analysis_results'
OUTPUT_PATH = f'{PROJECT_PATH}/viral_hepatitis/comparison_results'
os.makedirs(OUTPUT_PATH, exist_ok=True)

# 패키지 설치
!pip install scanpy anndata scipy statsmodels seaborn -q

import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Step 1 결과 로드
print('📊 분석 결과 로딩...')
h5ad_files = [f for f in os.listdir(ANALYSIS_PATH) if f.startswith('hbv_tahoe_analysis')]
latest_file = sorted(h5ad_files)[-1]
print(f'   파일: {latest_file}')

adata = sc.read_h5ad(f'{ANALYSIS_PATH}/{latest_file}')

print(f'\n✅ STEP 1 완료!')
print(f'   세포 수: {adata.n_obs}')
print(f'   그룹: {adata.obs["hbv_status"].unique().tolist() if "hbv_status" in adata.obs.columns else "N/A"}')

In [ ]:
# @title STEP 2: 그룹별 세포 분포 비교
if 'hbv_status' not in adata.obs.columns:
    raise ValueError('⚠️ hbv_status 컬럼이 없습니다!')

print('📊 그룹별 UMAP 시각화')

# 그룹별 UMAP
groups = adata.obs['hbv_status'].unique()
n_groups = len(groups)

fig, axes = plt.subplots(1, n_groups, figsize=(5*n_groups, 5))
if n_groups == 1:
    axes = [axes]

for ax, group in zip(axes, groups):
    adata_subset = adata[adata.obs['hbv_status'] == group]
    sc.pl.umap(adata_subset, color='cell_type', ax=ax, show=False,
               title=f'{group} (n={len(adata_subset)})')

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/group_umap_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 2 완료!')

In [ ]:
# @title STEP 3: 세포 유형별 비율 분석
print('📊 세포 유형별 비율 분석')

# 그룹별 세포 유형 비율 계산
cell_type_counts = adata.obs.groupby(['hbv_status', 'cell_type']).size().unstack(fill_value=0)
cell_type_ratios = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0) * 100

print('\n📋 그룹별 세포 유형 비율 (%):')
print(cell_type_ratios.round(1))

# 시각화: Stacked Bar Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 1. Stacked bar
cell_type_ratios.plot(kind='bar', stacked=True, ax=axes[0], colormap='tab10')
axes[0].set_title('Cell Type Distribution by HBV Status')
axes[0].set_xlabel('HBV Status')
axes[0].set_ylabel('Percentage (%)')
axes[0].legend(bbox_to_anchor=(1.02, 1), loc='upper left')
axes[0].tick_params(axis='x', rotation=45)

# 2. Heatmap
sns.heatmap(cell_type_ratios, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[1])
axes[1].set_title('Cell Type Percentage by Group')

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/cell_type_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 3 완료!')

In [ ]:
# @title STEP 4: 탈진 마커 발현 분석 (T세포 탈진)
print('🔬 T세포 탈진 마커 분석')

# 탈진 관련 마커
exhaustion_markers = {
    'PD-1': 'PDCD1',
    'TIM-3': 'HAVCR2',
    'LAG-3': 'LAG3',
    'CTLA-4': 'CTLA4',
    'TIGIT': 'TIGIT'
}

# Treg 마커
treg_markers = ['FOXP3', 'IL2RA', 'IKZF2']

# 사용 가능한 마커만 선택
available_exhaustion = {k: v for k, v in exhaustion_markers.items() if v in adata.var_names}
available_treg = [g for g in treg_markers if g in adata.var_names]

print(f'   탈진 마커 (사용 가능): {list(available_exhaustion.keys())}')
print(f'   Treg 마커 (사용 가능): {available_treg}')

if available_exhaustion:
    # 그룹별 탈진 마커 발현
    fig, axes = plt.subplots(1, len(available_exhaustion), figsize=(4*len(available_exhaustion), 5))
    if len(available_exhaustion) == 1:
        axes = [axes]
    
    for ax, (marker_name, gene) in zip(axes, available_exhaustion.items()):
        sc.pl.violin(adata, gene, groupby='hbv_status', ax=ax, show=False)
        ax.set_title(f'{marker_name} ({gene})')
        ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_PATH}/exhaustion_markers.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print('⚠️ 탈진 마커가 데이터에 없습니다.')

print(f'\n✅ STEP 4 완료!')

In [ ]:
# @title STEP 5: 차별 발현 유전자 분석 (DEG)
print('🧬 차별 발현 유전자 분석')

# Healthy vs HBV 비교
if 'Healthy' in adata.obs['hbv_status'].values:
    # Healthy와 비교
    sc.tl.rank_genes_groups(adata, groupby='hbv_status', reference='Healthy', method='wilcoxon')
    
    # 결과 시각화
    sc.pl.rank_genes_groups(adata, n_genes=10, sharey=False)
    plt.savefig(f'{OUTPUT_PATH}/deg_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # 상위 DEG 목록
    print('\n📋 그룹별 상위 차별 발현 유전자:')
    for group in adata.obs['hbv_status'].unique():
        if group != 'Healthy':
            genes = sc.get.rank_genes_groups_df(adata, group=group).head(5)
            print(f'\n{group} vs Healthy:')
            print(genes[['names', 'logfoldchanges', 'pvals_adj']].to_string(index=False))
else:
    print('⚠️ Healthy 그룹이 없어 첫 번째 그룹을 기준으로 비교')
    sc.tl.rank_genes_groups(adata, groupby='hbv_status', method='wilcoxon')
    sc.pl.rank_genes_groups(adata, n_genes=10, sharey=False)

print(f'\n✅ STEP 5 완료!')

In [ ]:
# @title STEP 6: 통계적 검정
print('📊 통계적 유의성 검정')

# 그룹별 세포 유형 비율에 대한 Chi-square test
print('\n1️⃣ Chi-square test (세포 유형 분포):')
contingency_table = adata.obs.groupby(['hbv_status', 'cell_type']).size().unstack(fill_value=0)
chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
print(f'   Chi-square: {chi2:.2f}')
print(f'   p-value: {p_value:.2e}')
print(f'   유의성: {"✅ 유의함" if p_value < 0.05 else "❌ 유의하지 않음"}')

# 각 세포 유형별 비율 비교
print('\n2️⃣ 세포 유형별 그룹 간 비교 (Fisher exact test):')
results = []
for cell_type in adata.obs['cell_type'].unique():
    groups_data = {}
    for group in adata.obs['hbv_status'].unique():
        subset = adata[adata.obs['hbv_status'] == group]
        ratio = (subset.obs['cell_type'] == cell_type).mean() * 100
        groups_data[group] = ratio
    
    results.append({'Cell Type': cell_type, **groups_data})

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

# 결과 저장
results_df.to_csv(f'{OUTPUT_PATH}/cell_type_ratios_by_group.csv', index=False)

print(f'\n✅ STEP 6 완료!')

In [ ]:
# @title STEP 7: 종합 결과 저장
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Summary 생성
summary = f"""
================================================================================
🦠 HBV 면역세포 비교 분석 결과
================================================================================

📅 분석 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

📊 데이터 요약:
   - 총 세포 수: {adata.n_obs}
   - 그룹 수: {adata.obs['hbv_status'].nunique()}
   - 세포 유형 수: {adata.obs['cell_type'].nunique()}

📋 그룹별 세포 수:
{adata.obs['hbv_status'].value_counts().to_string()}

🔬 주요 발견:
   - 세포 분포 chi-square p-value: {p_value:.2e}
   - 유의성: {'유의함 (p < 0.05)' if p_value < 0.05 else '유의하지 않음'}

📁 저장된 파일:
   - group_umap_comparison.png
   - cell_type_distribution.png
   - exhaustion_markers.png
   - deg_analysis.png
   - cell_type_ratios_by_group.csv

================================================================================
"""

print(summary)

# Summary 저장
with open(f'{OUTPUT_PATH}/analysis_summary_{timestamp}.txt', 'w') as f:
    f.write(summary)

# AnnData 저장
adata.write_h5ad(f'{OUTPUT_PATH}/hbv_comparison_results_{timestamp}.h5ad')

print('\n🎉 모든 분석이 완료되었습니다!')
print(f'📁 결과 폴더: {OUTPUT_PATH}')

---
## 📚 결과 해석 가이드

### 면역 회피 (Immune Escape) 지표:
| 지표 | 정상 | HBV 관용기 | 의미 |
|------|------|-----------|------|
| CD8+ T세포 비율 | 높음 | ↓ 감소 | 세포독성 면역 저하 |
| PD-1+ 세포 | 낮음 | ↑ 증가 | T세포 탈진 |
| Treg 비율 | 낮음 | ↑ 증가 | 면역 억제 |
| NK 세포 | 높음 | ↓ 감소 | 선천 면역 저하 |

### 다음 단계:
1. 실제 HBV 환자 데이터로 분석 반복
2. 치료 반응 예측 모델 개발
3. 간 조직 vs PBMC 비교 (GSE182159)